In [1]:
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from options.train_options import TrainOptions
from options.test_options import TestOptions
import os


import torch.nn as nn
import numpy as np
from data import CreateSrcDataLoader
from data import CreateTrgDataLoader
from model import CreateModel
# from model import CreateDiscriminator
from utils.timer import Timer
import tensorboardX

from matplotlib import pyplot as plt
import random
from PIL import Image
from skimage.transform import resize
from IPython.display import clear_output
# import evaluation
import os.path as osp
import collections
from torch.utils import data
from PIL import Image
import tqdm
import time
import run_time_evaluation
from IPython.display import clear_output


Init settings

In [2]:
root_base = '/home/olektra_gpu/Desktop/olektra projects/ECCV'                   #Root directory
sorted_list = root_base +'/dataset/cityscapes_list/sorted.txt'         # Final sorted list's path
data_gen_list = root_base +'/dataset/cityscapes_list/data_gen_list.txt'# data generated list's path
generated_data = root_base +'/dataset/generated_data/'                 # generated data path
generated_data_path = root_base +"/dataset/generated_data/"


select_model = 'DeepLab'                                                   # 'VGG' or 'DeepLab'
source_data = 'synthia'                                                   # 'synthia' or 'gta5'
model_pick = 'init'                                                    #'init' or 'annex'
model_weights = 'syn_2_city_deeplab'                                 # weights file's name


IMG_W = 1024                                                           #Image width
IMG_H = 512                                                            #Image height

In [3]:
eval_list = root_base +'/dataset/cityscapes_list/eval_.txt'

# Utils


In [4]:
file_name_save = root_base + '/snapshots/mIoU.txt'
def print_args(args):
    message = ''
    message += '----------------- Options ---------------\n'
    for k, v in sorted(vars(args).items()):
        comment = ''
        message += '{:>25}: {:<30}{}\n'.format(str(k), str(v), comment)
    message += '--------------------------------------'
   

    # save to the disk
    
    with open(file_name_save, 'a') as args_file:
        args_file.write(message)
        args_file.write('\n')   

In [5]:

def change_args(args):
    args.model =select_model
    args.source=source_data
    args.data_dir=root_base + '/dataset/'+args.source
    args.data_list = root_base+'/dataset/'+args.source+'_list/train.txt'
    args.data_list_target = root_base +'/dataset/cityscapes_list/train.txt'
    args.restore_from   = root_base + '/init_models/' + model_pick+ '/' + model_weights   #init_weights
    args.batch_size = 1
    args.weight_decay = 0.0005
    
    

In [6]:
classes = ['road' , 'side walk', 'building' , 'wall', 'fence', 'pole', 'trafic lights', 'trafic sign', 'vegitation', 'terrain', 'sky', 'person', 'rider', 'car', 'truck','bus', 'train', 'motorcycle', 'bicycle']

In [7]:
def keras_out(inp):
    inp = np.rollaxis(inp,axis =-1)
    inp = np.rollaxis(inp,axis =-1)
    inp = inp.reshape(1,inp.shape[0],inp.shape[1],inp.shape[2])
    
    return inp

In [8]:
def Plot_img(out):
    plt.imshow(out.reshape(IMG_H,IMG_W,3),interpolation='nearest')
    plt.grid(False)
    plt.axis('off')
    plt.show()

In [9]:
def self_entropy(pred, epsilon=1e-12):
    pred = pred[0]
    p = pred * np.log(pred+ epsilon)
    map_ = -np.sum(p, -1)
    
    return map_






Load areas




In [10]:
# area_save_path = root_base + "/areas_and_spatial_priors/area_classes_s.npy"
# class_mArea_man = np.load(area_save_path)
# arr=np.ones((19))
# arr[0:19] = class_mArea_man
# weights =  (arr)
# weight_mat = np.zeros((512,1024,19))
# for i in range(512):
#     for j in range(1024):
#         weight_mat[i][j] = weights
        
# weight_mat = np.rollaxis(weight_mat,axis = -1)
# print(weight_mat.shape)


In [11]:
# weights

Load priors of source

In [12]:
# save_pth = root_base + "/priors_of_source/priors_gta5.npy"
# priors_ = np.load(save_pth)

In [13]:
# priors__ = resize(priors_, (19,1024, 2048), anti_aliasing=True)

In [14]:
# from scipy.ndimage.filters import gaussian_filter
# source_priros= np.zeros((19,1024,2048))
# for j in range(19):
    
#     source_priros[j] = gaussian_filter(priors__[j], sigma=70)

In [15]:
# fig = plt.figure(figsize = (5,4))
# columns = 5
# rows = 4

# for i in range(1, columns * rows ):
#     fig.add_subplot(rows, columns, i)
#     fig.set_size_inches(18.5,10.5)
#     plt.imshow(source_priros[i-1],interpolation = 'nearest')
#     plt.grid(False)
#     plt.title(classes[i-1])
    
# plt.show()

# Model

In [16]:
def ent_normalization(SE,Y_pre):
    tY_pre = Y_pre[0,:,:,:]
    labs = np.argmax(tY_pre, axis=-1)
    labs1 = labs.flatten()
    se1 = SE.flatten()

    uniq = np.unique(labs1)

    for un in range(uniq.shape[0]):

        t_se = se1[labs1==uniq[un]]
        
        
        
        
        
        
        
        
        

        t_se1 = (t_se - t_se.min() )/(t_se.max()-t_se.min() )

        se1[labs1==uniq[un]] = t_se1

    SE = se1.reshape(SE.shape)
    return SE

In [17]:
class model_init():
    def __init__(self):
        opt = TrainOptions()
        args = opt.initialize()
        
        
        change_args(args)
        _t = {'iter time' : Timer()}
        model_name = args.source + '_to_' + args.target

        if not os.path.exists(args.snapshot_dir):
            os.makedirs(args.snapshot_dir)   
            os.makedirs(os.path.join(args.snapshot_dir, 'logs'))
  
        
        
        model, optimizer = CreateModel(args)
 

        self.args =args
        self.model_name =model_name
        self.model = model
        self.optimizer = optimizer
        self.opt = opt
        
    

In [18]:
IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)

In [19]:
def load_single_image(name):
    
    image = Image.open(osp.join(root_base + '/dataset/cityscapes/', "leftImg8bit/%s/%s" % ('train', name))).convert('RGB')
    # resize
    image = image.resize((IMG_W,IMG_H), Image.BICUBIC)

    rl_image = np.asarray(image, np.float32)
    rl_image_rgb = rl_image.copy()


    size = rl_image.shape
    image = rl_image[:, :, ::-1]  # change to BGR
    image -= IMG_MEAN
    image = image.transpose((2, 0, 1))
    

    return image.copy()

In [20]:

epsilon = 1e-12
cnt_img =0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Base_Adaptation():
    def __init__(self,model_loader):
        self.args =model_loader.args
        self.model_name =model_loader.model_name
        self.model = model_loader.model
        self.optimizer = model_loader.optimizer
        self.cnt_img = 0
        self.entropy_th_class = np.ones((1,19)) #* self.args.entropy_th
        
    def sorting(self):
        print("*********** Finding most confident samples using",self.args.sorting_method," ***********")
        if(self.args.sorting_method == 'class_wise'):
            self.class_wise_sort()
        else:
            self.global_sort()
        print("***************END********************")
            
            
    
    def class_wise_sort(self):
        self.args.data_label_folder_target = None 
        self.args.shuffel_ = False
        self.args.data_dir_target = root_base +'/dataset/cityscapes'
        self.args.data_list_target = root_base +'/dataset/cityscapes_list/train.txt'
        self.args.num_steps = self.args.total_no_of_target
        self.args.batch_size = 1
        self.model.eval()
        self.model.cuda()    
        targetloader = CreateTrgDataLoader(self.args)
        
        data_save_class = {classes[0]:[],classes[1]:[],classes[2]:[],classes[3]:[],classes[4]:[],classes[5]:[],classes[6]:[],classes[7]:[],classes[8]:[],classes[9]:[],classes[10]:[],classes[11]:[],classes[12]:[],classes[13]:[],classes[14]:[],classes[15]:[],classes[16]:[],classes[17]:[],classes[18]:[],"f_name_"+classes[0]:[],"f_name_"+classes[1]:[],"f_name_"+classes[2]:[],"f_name_"+classes[3]:[],"f_name_"+classes[4]:[],"f_name_"+classes[5]:[],"f_name_"+classes[6]:[],"f_name_"+classes[7]:[],"f_name_"+classes[8]:[],"f_name_"+classes[9]:[],"f_name_"+classes[10]:[],"f_name_"+classes[11]:[],"f_name_"+classes[12]:[],"f_name_"+classes[13]:[],"f_name_"+classes[14]:[],"f_name_"+classes[15]:[],"f_name_"+classes[16]:[],"f_name_"+classes[17]:[],"f_name_"+classes[18]:[]}
        data_save_class_sorted = {classes[0]:[],classes[1]:[],classes[2]:[],classes[3]:[],classes[4]:[],classes[5]:[],classes[6]:[],classes[7]:[],classes[8]:[],classes[9]:[],classes[10]:[],classes[11]:[],classes[12]:[],classes[13]:[],classes[14]:[],classes[15]:[],classes[16]:[],classes[17]:[],classes[18]:[],"f_name_"+classes[0]:[],"f_name_"+classes[1]:[],"f_name_"+classes[2]:[],"f_name_"+classes[3]:[],"f_name_"+classes[4]:[],"f_name_"+classes[5]:[],"f_name_"+classes[6]:[],"f_name_"+classes[7]:[],"f_name_"+classes[8]:[],"f_name_"+classes[9]:[],"f_name_"+classes[10]:[],"f_name_"+classes[11]:[],"f_name_"+classes[12]:[],"f_name_"+classes[13]:[],"f_name_"+classes[14]:[],"f_name_"+classes[15]:[],"f_name_"+classes[16]:[],"f_name_"+classes[17]:[],"f_name_"+classes[18]:[]}
    
        
        
        for index, batch in tqdm.tqdm(enumerate(targetloader)):
            
            image, _, name ,__= batch
            fn = name[0]
            output = self.model(Variable(image).cuda())
            output = nn.functional.softmax(output, dim=1)
            output=output.cpu().data[0].numpy()
            y_pred = keras_out(output)
            
            
            
            MP_max = np.max(y_pred,axis = -1)[0]
            LP_argmax = np.argmax(y_pred,axis = -1)[0]
            
      
            for c in range(len(classes) ):
                M_c = MP_max[LP_argmax == c]
                if(M_c.any()):
                    data_save_class['f_name_'+classes[c]].append(fn)
                    data_save_class[classes[c]].append(np.mean(M_c))
        

        f = open(sorted_list,"w+") 
        f_ev = open(eval_list,"w+") 
        self.cnt_img=0
        over_all_list = []
        for c in range(len(classes) ):      
            
            if(source_data == 'synthia'):
                if(c !=9 and c !=14 and c !=16):
                    max_prob_sorted_ascending, f_name_arrays = zip(*sorted(zip(data_save_class[classes[c]], data_save_class['f_name_'+classes[c]])))
                    f_name_arrays = f_name_arrays[::-1]
                    max_prob_sorted_ascending = max_prob_sorted_ascending[::-1]

                    data_save_class_sorted[classes[c]] = max_prob_sorted_ascending
                    data_save_class_sorted['f_name_'+classes[c]] = f_name_arrays
                    select_imgs =  int(len(f_name_arrays) * (self.args.p/19))

                    selct = f_name_arrays[0:select_imgs]



                    ######Dynamic threshold of each class##############

                    f_n = f_name_arrays[select_imgs]
                    img= load_single_image(f_n)
                    img = img.reshape(1,3,IMG_H,IMG_W)
                    img = torch.from_numpy(img).float().to(device)
                    out_ = self.model(Variable(img).cuda())
                    out_ = nn.functional.softmax(out_, dim=1)
                    out_=out_.cpu().data[0].numpy()
                    y_pred = keras_out(out_)


                    Y_argmax = np.argmax(y_pred,axis = -1)[0]
                    SE_main = self_entropy(y_pred)

                    if(self.args.entropy_normalization):
                        SE_main = ent_normalization(SE_main,y_pred)

                    class_se = SE_main[Y_argmax == c]
                    mean_class_se = np.mean(class_se) * self.args.ent_th_on_class_gain


                    self.entropy_th_class[0][c] = mean_class_se

                    ##############################################

                    over_all_list += selct
            else:
                max_prob_sorted_ascending, f_name_arrays = zip(*sorted(zip(data_save_class[classes[c]], data_save_class['f_name_'+classes[c]])))
                f_name_arrays = f_name_arrays[::-1]
                max_prob_sorted_ascending = max_prob_sorted_ascending[::-1]

                data_save_class_sorted[classes[c]] = max_prob_sorted_ascending
                data_save_class_sorted['f_name_'+classes[c]] = f_name_arrays
                select_imgs =  int(len(f_name_arrays) * (self.args.p/19))

                selct = f_name_arrays[0:select_imgs]
                f_ev.write(classes[c]+'\n')
                for loop in f_name_arrays[0:10]:
                    f_ev.write(loop+'\n')
                
                    
                

                ######Dynamic threshold of each class##############

                f_n = f_name_arrays[select_imgs]
                img= load_single_image(f_n)
                img = img.reshape(1,3,IMG_H,IMG_W)
                img = torch.from_numpy(img).float().to(device)
                out_ = self.model(Variable(img).cuda())
                out_ = nn.functional.softmax(out_, dim=1)
                out_=out_.cpu().data[0].numpy()
                y_pred = keras_out(out_)


                Y_argmax = np.argmax(y_pred,axis = -1)[0]
                SE_main = self_entropy(y_pred)

                if(self.args.entropy_normalization):
                    SE_main = ent_normalization(SE_main,y_pred)

                class_se = SE_main[Y_argmax == c]
                mean_class_se = np.mean(class_se) * self.args.ent_th_on_class_gain


                self.entropy_th_class[0][c] = mean_class_se

                ##############################################

                over_all_list += selct
            
        over_all_list = list(dict.fromkeys(over_all_list))
        for loop in over_all_list:
            f.write(loop+'\n')
            self.cnt_img +=1
    

        
        
        
    
    def global_sort(self):
        self.args.data_label_folder_target = None 
        self.args.shuffel_ = False
        self.args.data_dir_target = root_base +'/dataset/cityscapes'
        self.args.data_list_target = root_base +'/dataset/cityscapes_list/train.txt'
        self.args.num_steps = self.args.total_no_of_target
        self.args.batch_size = 1
        self.model.eval()
        self.model.cuda()    
        targetloader = CreateTrgDataLoader(self.args)
        data_save_global = {"f_name":[],"max_prob_values":[]} 
        
        for index, batch in tqdm.tqdm(enumerate(targetloader)):
            
            image, _, name ,__= batch
            fn = name[0]
#             print(fn)
            output = self.model(Variable(image).cuda())
            output = nn.functional.softmax(output, dim=1)
            output=output.cpu().data[0].numpy()
            y_pred = keras_out(output)

            MP_max = np.max(y_pred,axis = -1)[0]
          
            data_save_global['f_name'].append(fn)
            data_save_global['max_prob_values'].append(np.mean(MP_max))

        max_prob_sorted_ascending, f_name_arrays = zip(*sorted(zip(data_save_global['max_prob_values'], data_save_global['f_name'])))
        f_name_arrays = f_name_arrays[::-1]
        max_prob_sorted_ascending = max_prob_sorted_ascending[::-1]

        data_save_global['f_name'] = f_name_arrays
        data_save_global['max_prob_values'] = max_prob_sorted_ascending
        
        f = open(sorted_list,"w+")
        for loop in data_save_global['f_name']:
            f.write(loop+'\n')
            
        
          
    
    def save_data(self,patch_x, patch_y, path,i,map_):
      
        Y = np.argmax(patch_y,axis = -1)
        if(map_ != 'a'):
            Y[map_ == 0] = 255
        plt.imsave( path +"images/"+ str(i)+'.png',patch_x)  #p_small_data
        Y=np.asarray(Y,dtype=np.uint8)
        
       
        Y=Image.fromarray(Y,mode = 'L')

        
        Y.save(path+ "labels/"+str(i)+'.png')
        
    def Data_Generate(self):
        if(self.args.patch_select_method == 'smart'):
            self.data_generate_smart()
        else:
            self.data_generator()
            
    def data_generate_smart(self):
        self.args.data_label_folder_target = None
        if(self.args.sorting_method == 'class_wise'):
            stop_point = self.cnt_img
        else:
            stop_point = int(self.args.p * self.args.total_no_of_target)
            
       
        self.args.shuffel_ = False
        self.args.data_dir_target = root_base +'/dataset/cityscapes'
        self.args.data_list_target = sorted_list
        
        self.args.batch_size = 1
        self.model.eval()
        self.model.cuda()    
        targetloader = CreateTrgDataLoader(self.args)
        
        count = 0
        
        f = open(data_gen_list,"w+")
        print("***********Generating Data smart***********")
        print("Total it : ",stop_point+1)
        for index, batch in tqdm.tqdm(enumerate(targetloader)):
            
            if(index > stop_point):
                break
            
            image, _, name, rl_img = batch
            fn = name[0]
            output = self.model(Variable(image).cuda())
            output = nn.functional.softmax(output, dim=1)
            rl_img = rl_img.cpu().data[0].numpy()/255.0
            output=output.cpu().data[0].numpy()
            
            image=image.cpu().data[0].numpy()
            image = np.rollaxis(image, axis = -1)
            image = np.rollaxis(image, axis = -1)
            
            if(self.args.balance_on_priors):
                wei = (1 - weight_mat )
                output = np.multiply(output,wei)
                    
            
            
            Y_pred = keras_out(output)
            
            ################################################
            map_ = np.zeros((IMG_H,IMG_W))
            SE = self_entropy(Y_pred)
            
            if(self.args.entropy_normalization):
                SE = ent_normalization(SE,Y_pred)
            
            if(self.args.ent_th_on_class):
                
                for il in range(IMG_H):
                    for ij in range(IMG_W):
                        if(SE[il][ij] >= self.entropy_th_class[0][np.argmax(Y_pred[0][il][ij],axis = -1)]):
                            map_[il][ij] =  0
                        else:
                            map_[il][ij] =  1
                                    
            else:
            
                map_[SE >= self.args.entropy_th] =  0
                map_[SE < self.args.entropy_th] =  1
            cnt_flag = 0
            
            ###################save orignal#################
            
#             f.write(str(count)+".png"+'\n')
#             self.save_data(rl_img, Y_pred.reshape(IMG_H,IMG_W,19), generated_data,(count),'a')
#             count +=1
            ##########################################
            
            
            for patch in range(self.args.total_patches_in_smart):

                point_X_s = self.args.patch_size[1]
                point_Y_s = self.args.patch_size[0]

                
                point_X = random.randint(0, IMG_W-point_X_s)
                point_Y = random.randint(0, IMG_H - point_Y_s)
                
                #########################################
                x_mean_patch =  image[ point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s,3)
                map_patch = map_[point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s)
   
                x_PATCH =  rl_img[ point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s,3)
                y_PATCH =  Y_pred[0, point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(1,point_Y_s,point_X_s,19)
                
            
                y_PATCH = np.rollaxis(y_PATCH[0], axis = -1).reshape(1,19,point_Y_s,point_X_s)
                y_PATCH = torch.from_numpy(y_PATCH).float().to(device)
                Y_Patch_resize = nn.functional.upsample(y_PATCH, (IMG_H, IMG_W), mode='bilinear', align_corners=True).cpu().data[0].numpy()
                Y_Patch_resize = np.rollaxis(Y_Patch_resize, axis = -1)
                Y_Patch_resize = np.rollaxis(Y_Patch_resize, axis = -1).reshape(1,IMG_H,IMG_W,19)

            
            
                
                #############smart patch seperateeeeeer#########
                x_mean_patch = np.rollaxis(x_mean_patch, axis = -1).reshape(1,3,point_Y_s,point_X_s)
                x_mean_patch = torch.from_numpy(x_mean_patch).float().to(device)
                x_mean_res = nn.functional.upsample(x_mean_patch, (IMG_H, IMG_W), mode='bilinear', align_corners=True)
                
                
                output = self.model(Variable(x_mean_res).cuda())
                output = nn.functional.softmax(output, dim=1)
                output=output.cpu().data[0].numpy()
                y_patch_out = keras_out(output)
                
                SE_main = self_entropy(Y_Patch_resize)
                SE_patch = self_entropy(y_patch_out)
                
                diff = (np.mean(SE_patch) - np.mean(SE_main))
                
                ##################################
                
                if(diff >= self.args.entropy_diff_th):
                    #####################code new ################
                    x_PATCH = np.rollaxis(x_PATCH, axis = -1).reshape(1,3,point_Y_s,point_X_s)
                    x_PATCH = torch.from_numpy(x_PATCH).float().to(device)

                    
                    map_patch = map_patch.reshape(1,1,point_Y_s,point_X_s)
                    map_patch = torch.from_numpy(map_patch).float().to(device)

                    X_Patch_resize = nn.functional.upsample(x_PATCH, (IMG_H, IMG_W), mode='bilinear', align_corners=True).cpu().data[0].numpy()
                    X_Patch_resize = np.rollaxis(X_Patch_resize, axis = -1)
                    X_Patch_resize = np.rollaxis(X_Patch_resize, axis = -1)


                    Y_Patch_resize = nn.functional.upsample(y_PATCH, (IMG_H, IMG_W), mode='bilinear', align_corners=True).cpu().data[0].numpy()
                    Y_Patch_resize = np.rollaxis(Y_Patch_resize, axis = -1)
                    Y_Patch_resize = np.rollaxis(Y_Patch_resize, axis = -1).reshape(1,IMG_H,IMG_W,19)


                    map_patch_re = nn.functional.upsample(map_patch, (IMG_H, IMG_W), mode='bilinear', align_corners=True).cpu().data[0].numpy()
                    map_patch_re = map_patch_re[0]

                    #############################################
                    cnt_flag +=1
                    f.write(str(count+cnt_flag)+".png"+'\n')
                    self.save_data(X_Patch_resize, Y_Patch_resize.reshape(IMG_H,IMG_W,19), generated_data,(count+cnt_flag),map_patch_re)
                    
                
                
                if(cnt_flag >= self.args.no_of_patches_per_image):
                    break
            count+=self.args.no_of_patches_per_image + 1
            
        print("*****************END****************")
        
        
    def data_generator(self):
        self.args.data_label_folder_target = None
        if(self.args.sorting_method == 'class_wise'):
            stop_point = self.cnt_img
        else:
            stop_point = int(self.args.p * self.args.total_no_of_target)
            
       
        self.args.shuffel_ = False
        self.args.data_dir_target = root_base +'/dataset/cityscapes'
        self.args.data_list_target = sorted_list
        
        self.args.batch_size = 1
        self.model.eval()
        self.model.cuda()    
        targetloader = CreateTrgDataLoader(self.args)
        
        count = 0
        
        f = open(data_gen_list,"w+")
        print("***********Generating Data Random***********")
        print("Total it : ",stop_point+1)
        for index, batch in tqdm.tqdm(enumerate(targetloader)):
            
            if(index > stop_point):
                break
            
            image, _, name, rl_img = batch
            fn = name[0]
            output = self.model(Variable(image).cuda())
            output = nn.functional.softmax(output, dim=1)
            rl_img = rl_img.cpu().data[0].numpy()/255.0
            output=output.cpu().data[0].numpy()
             
            
            if(self.args.balance_on_priors):
                wei = (1 - weight_mat )
                output = np.multiply(output,wei)
                    
            
            
            Y_pred = keras_out(output)
            
            ###############################################
            map_ = np.zeros((IMG_H,IMG_W))
            SE = self_entropy(Y_pred)
            
            if(self.args.entropy_normalization):
                SE = ent_normalization(SE,Y_pred)
            
            if(self.args.ent_th_on_class):
                
                for il in range(IMG_H):
                    for ij in range(IMG_W):
                        
                        if(SE[il][ij] >= self.entropy_th_class[0][np.argmax(Y_pred[0][il][ij],axis = -1)]):
                            map_[il][ij] =  0
                        else:
                            map_[il][ij] =  1
                                    
            else:
            
                map_[SE >= self.args.entropy_th] =  0
                map_[SE < self.args.entropy_th] =  1
            ###################save orignal#################
            
#             f.write(str(count)+".png"+'\n')
#             self.save_data(rl_img, Y_pred.reshape(IMG_H,IMG_W,19), generated_data,(count),map_)
#             count +=1
            ##########################################
            
            if(self.args.shrink_image):
                f.write(str(count)+".png"+'\n')
                IMG_SS = np.zeros((IMG_H,IMG_W,3))
                Y_PRE_SS = np.zeros((1,IMG_H,IMG_W,19))
                MAP_SS= np.zeros((IMG_H,IMG_W))
                
                img_ss = resize(rl_img, (self.args.patch_size[0], self.args.patch_size[1]), anti_aliasing=True)
                y_pred_ss = resize(Y_pred, (1,self.args.patch_size[0], self.args.patch_size[1]), anti_aliasing=False)
                
                point_X_s = self.args.patch_size[1]
                point_Y_s = self.args.patch_size[0]


#                 point_X = random.randint(0, IMG_W-point_X_s)
#                 point_Y = random.randint(0, IMG_H - point_Y_s)
                
                point_X = 256
                point_Y = 128
                

                IMG_SS[point_Y:point_Y+self.args.patch_size[0] , point_X:point_X+self.args.patch_size[1]]= img_ss
                Y_PRE_SS[0,point_Y:point_Y+self.args.patch_size[0] , point_X:point_X+self.args.patch_size[1]]= y_pred_ss
                MAP_SS[point_Y:point_Y+self.args.patch_size[0] , point_X:point_X+self.args.patch_size[1]] =1
                
                self.save_data(IMG_SS, Y_PRE_SS.reshape(IMG_H,IMG_W,19), generated_data,(count),MAP_SS)
                count +=1
                
            for patch in range(self.args.no_of_patches_per_image):
                
#                 point_X_s = 256 * (patch +1)
#                 point_Y_s = int(point_X_s/2)
                
#                 point_X = 512 - int(point_X_s/2)
#                 point_Y = 256 - int(point_Y_s/2)
                
                if(self.args.patch_size[1] == 1024):
                    X_Patch_resize = rl_img
                    Y_Patch_resize = Y_pred
                    map_patch_re = map_
                    
                else:

                    point_X_s = self.args.patch_size[1]
                    point_Y_s = self.args.patch_size[0]


                    point_X = random.randint(0, IMG_W-point_X_s)
                    point_Y = random.randint(0, IMG_H - point_Y_s)



                    x_PATCH =  rl_img[ point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s,3)
                    y_PATCH =  Y_pred[0, point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(1,point_Y_s,point_X_s,19)
                    map_patch = map_[point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s)
                    
                    
                    ################resize on pytorch ###############
                    x_PATCH = np.rollaxis(x_PATCH, axis = -1).reshape(1,3,point_Y_s,point_X_s)
                    x_PATCH = torch.from_numpy(x_PATCH).float().to(device)
                    
                    y_PATCH = np.rollaxis(y_PATCH[0], axis = -1).reshape(1,19,point_Y_s,point_X_s)
                    y_PATCH = torch.from_numpy(y_PATCH).float().to(device)
                    
                    map_patch = map_patch.reshape(1,1,point_Y_s,point_X_s)
                    map_patch = torch.from_numpy(map_patch).float().to(device)
                    
                    X_Patch_resize = nn.functional.upsample(x_PATCH, (IMG_H, IMG_W), mode='bilinear', align_corners=True).cpu().data[0].numpy()
                    X_Patch_resize = np.rollaxis(X_Patch_resize, axis = -1)
                    X_Patch_resize = np.rollaxis(X_Patch_resize, axis = -1)
                    
                    
                    Y_Patch_resize = nn.functional.upsample(y_PATCH, (IMG_H, IMG_W), mode='bilinear', align_corners=True).cpu().data[0].numpy()
                    Y_Patch_resize = np.rollaxis(Y_Patch_resize, axis = -1)
                    Y_Patch_resize = np.rollaxis(Y_Patch_resize, axis = -1).reshape(1,IMG_H,IMG_W,19)
                    
                    
                    map_patch_re = nn.functional.upsample(map_patch, (IMG_H, IMG_W), mode='bilinear', align_corners=True).cpu().data[0].numpy()
                    map_patch_re = map_patch_re[0]
                   
                    #####################################
                    
                    
#                     X_Patch_resize = resize(x_PATCH, (IMG_H, IMG_W), anti_aliasing=False)
#                     Y_Patch_resize = resize(y_PATCH, (1,IMG_H, IMG_W), anti_aliasing=False)
#                     map_patch_re = resize(map_patch, (IMG_H, IMG_W), anti_aliasing=False)
                


                f.write(str(count+patch)+".png"+'\n')
                self.save_data(X_Patch_resize, Y_Patch_resize.reshape(IMG_H,IMG_W,19), generated_data,(count+patch),map_patch_re)
                
                
            count+=self.args.no_of_patches_per_image + 1
        print("*****************END****************")
            
             
    
                
    def gaussian(self, ins,mean, stddev):
  
        noise = Variable(ins.data.new(ins.size()).normal_(mean,stddev))
        
        return ins + noise
            
    def train_adp(self,round_):
        
        
        
        self.args.data_label_folder_target = root_base +'/dataset/generated_data/'   
        self.args.shuffel_ = True
        self.args.data_dir_target = root_base +'/dataset/generated_data/'
        self.args.data_list_target = data_gen_list
        self.args.batch_size = 1
        
        _t = {'iter time' : Timer()}
       
        if(self.args.sorting_method == 'class_wise'):
            self.args.num_steps = int(self.cnt_img * self.args.epoch_per_round * self.args.no_of_patches_per_image)
        else:
            
            self.args.num_steps = int(self.args.p*self.args.total_no_of_target*self.args.no_of_patches_per_image *self.args.epoch_per_round)
     
            
           
        sourceloader, targetloader = CreateSrcDataLoader(self.args), CreateTrgDataLoader(self.args)
        targetloader_iter, sourceloader_iter = iter(targetloader), iter(sourceloader)
        
        
        start_iter = 0
#         if self.args.restore_from is not None:
#             start_iter = int(self.args.restore_from.rsplit('/', 1)[1].rsplit('_')[1])
        
#         train_writer = tensorboardX.SummaryWriter(os.path.join(self.args.snapshot_dir, "logs", model_name))
    
#         bce_loss = torch.nn.BCEWithLogitsLoss()

        cudnn.enabled = True
        cudnn.benchmark = True
        self.model.train()
        self.model.cuda()

        loss = ['loss_seg_src', 'loss_seg_trg']
        _t['iter time'].tic()
        for i in range(start_iter, self.args.num_steps):
            
            self.model.adjust_learning_rate(self.args, self.optimizer, i)

            self.optimizer.zero_grad()

            src_img, src_lbl, _, _ = sourceloader_iter.next()
            
            src_img, src_lbl = Variable(src_img).cuda(), Variable(src_lbl.long()).cuda()
            src_seg_score = self.model(src_img, lbl=src_lbl)       
            loss_seg_src = self.model.loss
    
            loss_src = torch.mean(loss_seg_src)     
            ##############################
            loss_src.backward()
            
            
            
           
           
            
            trg_img, trg_lbl, _, _ = targetloader_iter.next()
            trg_img, trg_lbl = Variable(trg_img).cuda(), Variable(trg_lbl.long()).cuda()
#             trg_img = self.gaussian(trg_img,0, 0.02)
            trg_seg_score = self.model(trg_img, lbl=trg_lbl) 
 
            ############################
            loss_seg_trg = self.model.loss 
            
            ##########Focal loss############
            loss_trg_2 = torch.mean(loss_seg_trg)
            if(self.args.focal_flag):
                    #try to minimize segmentation loss


                pt = torch.exp(-loss_seg_trg)
                loss_trg =   loss_seg_trg  * (1-pt)**self.args.gamma
                trg_fcl = torch.mean(loss_trg)
            else:
                trg_fcl =0
            
            loss_trg =  0.1 *trg_fcl  + loss_trg_2
            
            
            ##############################
#             over_all_loss =   loss_trg     
  
            loss_trg.backward()
            
            
            

            

            
            src_seg_score, trg_seg_score = src_seg_score.detach(), trg_seg_score.detach()

            self.optimizer.step()

#             for m in loss:
#                 train_writer.add_scalar(m, eval(m), i+1)

            if (i+1) % 500 == 0:
                print ('taking snapshot ...')
                torch.save(self.model.state_dict(), os.path.join(self.args.snapshot_dir, '%s_' %(self.args.source+"_to_" +self.args.target )+"_"+ self.args.Expriment_name +str(round_) +'.pth' ))   

            if (i+1) % 100 == 0:
                _t['iter time'].toc(average=False)
                print ('[it %d][src seg loss %.4f][lr %.4f][%.2fs]' % \
                        (i + 1, loss_src.data, self.optimizer.param_groups[0]['lr']*10000, _t['iter time'].diff))
                if i + 1 > self.args.num_steps_stop:
                    print ('finish training')
                    break
                _t['iter time'].tic()

            
        
            
            
            
            
        
        



load init

In [21]:
model_initl = model_init()

Experiment's Settigns


In [22]:
model_initl.args.focal_flag = True
model_initl.args.shrink_image = False

model_initl.args.Expriment_name = 'Exp_Final_syntia_with_focal'     # name of an experiment
model_initl.args.p =0.1                # p is the portion for selecion of samples in target data ==> p*Total number of images
model_initl.args.no_of_patches_per_image = 4 # number of patchs extarct from each image
model_initl.args.gamma = 3                 # (1-prediction)**gamma  ==> focal loss
model_initl.args.learning_rate = 1e-12 # learning rate
# model_initl.args.entropy_th = 1.3            # Entropy map's threshold 
Rounds = int(1/model_initl.args.p)           # Total number of rounds 
model_initl.args.epoch_per_round = 2   # epoch each round

model_initl.args.sorting_method = 'class_wise'   # 'global' or 'class_wise'
model_initl.args.total_no_of_target = 2975         # Total number of target images


model_initl.args.patch_select_method = 'random'           # 'smart'  or 'random'
model_initl.args.total_patches_in_smart=None     # toatal number of patches in smart selection
model_initl.args.entropy_diff_th = None            # we select the specific patch which have entropy diff greater than this threshold
                                                          # seleection range  0.1  to 0.6

model_initl.args.balance_on_priors = False          # balance the prediction with priors info of area pred*(1-area)
model_initl.args.entropy_normalization = True      # entropy normalization or not
model_initl.args.patch_size = (256,512)

model_initl.args.ent_th_on_class = True        #Apply unique entropy threshold of each class seperately
model_initl.args.ent_th_on_class_gain = 1    # gain on class based entropy threshold 


print_args(model_initl.args)


In [23]:
model_initl.opt.print_options(model_initl.args)

----------------- Options ---------------
           Expriment_name: Exp_Final_syntia_with_focal   
        balance_on_priors: False                         
               batch_size: 1                             
                 data_dir: /home/olektra_gpu/Desktop/olektra projects/ECCV/dataset/synthia
          data_dir_target: /home/olektra_gpu/Desktop/olektra projects/ECCV/dataset/cityscapes
 data_label_folder_target: None                          
                data_list: /home/olektra_gpu/Desktop/olektra projects/ECCV/dataset/synthia_list/train.txt
         data_list_target: /home/olektra_gpu/Desktop/olektra projects/ECCV/dataset/cityscapes_list/train.txt
          ent_th_on_class: True                          
     ent_th_on_class_gain: 1                             
          entropy_diff_th: None                          
    entropy_normalization: True                          
          epoch_per_round: 2                             
               focal_flag: True     

In [24]:
Base = Base_Adaptation(model_initl)

In [25]:
# Base.args.p = 0.1
first_di = 0
record_entropy = []
record_mIoU_ = []
prev_entropy = 12345666555433232234234
for round_ in range(Rounds):
    print("############### Round ",round_," #####################")
    Base.sorting()
    Base.Data_Generate()
    Base.train_adp(round_)
    testing_entr, mIoU_ = run_time_evaluation.main(Base.model)
    record_entropy.append(np.mean(testing_entr))
    record_mIoU_.append(mIoU_)
    Base.args.p = Base.args.p+0.05
    print("#################### Round END #######################")
    diff = prev_entropy - np.mean(testing_entr)

    if(diff < 0):
        break
#     if( diff  <  1 and round_ > 0):
#         break
#     if(round_ == 1):
#         first_di = diff
#     if(first_di != 0 and diff > 0):
#         Base.args.learning_rate = Base.args.learning_rate * (diff/first_di)
    if(Base.args.p > 0.5):
        break
    clear_output()
    prev_entropy = np.mean(testing_entr)

    

############### Round  0  #####################
*********** Finding most confident samples using class_wise  ***********


0it [00:00, ?it/s]/home/olektra_gpu/.local/lib/python3.5/site-packages/torch/nn/functional.py:2416: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
2975it [33:59,  1.46it/s]
/home/olektra_gpu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


***************END********************
***********Generating Data Random***********
Total it :  165


0it [00:00, ?it/s]/home/olektra_gpu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:208: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
165it [15:58,  5.85s/it]

*****************END****************


/home/olektra_gpu/.local/lib/python3.5/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[it 100][src seg loss 0.1732][lr 0.0000][575.03s]
[it 200][src seg loss 0.1089][lr 0.0000][557.56s]
[it 300][src seg loss 0.0811][lr 0.0000][557.77s]
[it 400][src seg loss 0.0839][lr 0.0000][557.31s]
taking snapshot ...
[it 500][src seg loss 0.0638][lr 0.0000][557.96s]


KeyboardInterrupt: 

In [26]:
testing_entr, mIoU_ = run_time_evaluation.main(Base.model)


0it [00:00, ?it/s]
1it [00:01,  1.13s/it]
2it [00:02,  1.13s/it]
3it [00:03,  1.13s/it]
4it [00:04,  1.13s/it]
5it [00:05,  1.13s/it]
6it [00:06,  1.13s/it]
7it [00:07,  1.12s/it]
8it [00:09,  1.12s/it]
9it [00:10,  1.13s/it]
10it [00:11,  1.13s/it]
11it [00:12,  1.12s/it]
12it [00:13,  1.12s/it]
13it [00:14,  1.12s/it]
14it [00:15,  1.12s/it]
15it [00:16,  1.12s/it]
16it [00:17,  1.12s/it]
17it [00:19,  1.12s/it]
18it [00:20,  1.12s/it]
19it [00:21,  1.12s/it]
20it [00:22,  1.12s/it]
21it [00:23,  1.12s/it]
22it [00:24,  1.12s/it]
23it [00:25,  1.12s/it]
24it [00:26,  1.12s/it]
25it [00:28,  1.12s/it]
26it [00:29,  1.13s/it]
27it [00:30,  1.13s/it]
28it [00:31,  1.12s/it]
29it [00:32,  1.12s/it]
30it [00:33,  1.12s/it]
31it [00:34,  1.12s/it]
32it [00:35,  1.12s/it]
33it [00:37,  1.12s/it]
34it [00:38,  1.12s/it]
35it [00:39,  1.12s/it]
36it [00:40,  1.12s/it]
37it [00:41,  1.12s/it]
38it [00:42,  1.12s/it]
39it [00:43,  1.12s/it]
40it [00:44,  1.12s/it]
41it [00:45,  1.12s/it]
42it 

296it [05:31,  1.11s/it]
297it [05:32,  1.12s/it]
298it [05:33,  1.12s/it]
299it [05:34,  1.12s/it]
300it [05:36,  1.12s/it]
301it [05:37,  1.12s/it]
302it [05:38,  1.12s/it]
303it [05:39,  1.12s/it]
304it [05:40,  1.12s/it]
305it [05:41,  1.12s/it]
306it [05:42,  1.12s/it]
307it [05:43,  1.12s/it]
308it [05:44,  1.12s/it]
309it [05:46,  1.12s/it]
310it [05:47,  1.12s/it]
311it [05:48,  1.12s/it]
312it [05:49,  1.11s/it]
313it [05:50,  1.12s/it]
314it [05:51,  1.11s/it]
315it [05:52,  1.11s/it]
316it [05:53,  1.11s/it]
317it [05:54,  1.11s/it]
318it [05:56,  1.11s/it]
319it [05:57,  1.12s/it]
320it [05:58,  1.12s/it]
321it [05:59,  1.12s/it]
322it [06:00,  1.11s/it]
323it [06:01,  1.11s/it]
324it [06:02,  1.11s/it]
325it [06:03,  1.11s/it]
326it [06:05,  1.12s/it]
327it [06:06,  1.12s/it]
328it [06:07,  1.12s/it]
329it [06:08,  1.12s/it]
330it [06:09,  1.12s/it]
331it [06:10,  1.11s/it]
332it [06:11,  1.11s/it]
333it [06:12,  1.11s/it]
334it [06:13,  1.11s/it]
335it [06:15,  1.11s/it]


Num classes 19
===>road:	82.9
===>sidewalk:	43.08
===>building:	78.06
===>wall:	9.29
===>fence:	0.56
===>pole:	28.23
===>light:	9.13
===>sign:	14.45
===>vegetation:	77.02
===>terrain:	0.0
===>sky:	83.49
===>person:	58.13
===>rider:	25.9
===>car:	71.93
===>truck:	0.0
===>bus:	38.02
===>train:	0.0
===>motocycle:	29.43
===>bicycle:	31.16
===> mIoU19: 35.83
===> mIoU16: 42.55
===> mIoU13: 49.44


In [26]:
np.save(Base.args.Expriment_name+'entr_record.npy', np.array(record_entropy))
np.save(Base.args.Expriment_name+'Iou_record.npy', np.array(record_mIoU_))

In [27]:
print(record_entropy)

[238.91925, 211.53044, 201.52953, 181.916, 171.4761, 170.83728]


new flow

In [0]:
run_time_evaluation.main(Base.model,source_priros)

In [0]:
sourceloader, targetloader = CreateSrcDataLoader(Base.args), CreateTrgDataLoader(Base.args)
targetloader_iter, sourceloader_iter = iter(targetloader), iter(sourceloader)
       

In [0]:
src_img, src_lbl, _, _ = sourceloader_iter.next()
src_img=src_img.cpu().data[0].numpy()

In [0]:
plt.imshow(src_img.reshape(720,1280,3)/255,interpolation='nearest')
plt.grid(False)
plt.axis('off')
plt.show()

In [0]:
trg_img, trg_lbl, _, _ = targetloader_iter.next()

In [0]:
print(src_lbl.shape)



Analysis

In [0]:
def visualize(temp, plot=True):
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()
    for l in range(0,20):
        r[temp==l]=label_colours[l,0]
        g[temp==l]=label_colours[l,1]
        b[temp==l]=label_colours[l,2]

    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:,:,0] = (r/255.0)#[:,:,0]
    rgb[:,:,1] = (g/255.0)#[:,:,1]
    rgb[:,:,2] = (b/255.0)#[:,:,2]
    if plot:
        plt.imshow(rgb)
    else:
        return rgb

In [0]:
road = (128, 64,128)
side_walk = (232, 35,244)
building = (70, 70, 70)
wall = (102,102,156)
fence = (190,153,153)
pole = (153,153,153)
trafic_lights = (250,170, 30)
trafic_sign = (220,220,  0)
vegitation = (107,142, 35)
terrain = (152,251,152)
sky = (70,130,180)
person = (220, 20, 60)
rider = (255,  0,  0)
car = ( 0,  0,142)
truck = (0,  0, 70)
bus = ( 0, 60,100)
train = ( 0, 80,100)
motorcycle = (0,  0,230)
bicycle = (119, 11, 32)
else_ = ( 0,  0,  0)



label_colours = np.array([road, side_walk, building, wall,fence,
                          pole, trafic_lights , trafic_sign , vegitation,  terrain ,  sky,  person, rider , car , truck ,bus  , train , motorcycle, bicycle ,else_  ])



Entropy threshold for filtering of each class 

In [0]:
# import pandas as pd



# commutes = pd.DataFrame({'Entropy Threshold':Base.entropy_th_class.reshape(19)})   #Base.entropy_th_class.reshape(19)s

# # commutes.plot(grid=True, bins=20, rwidth=0.9,
# #                    color='#607c8e')
# # plt.title('Commute Times for 1,000 Commuters')
# # plt.xlabel('Counts')
# # plt.ylabel('Commute Time')
# # plt.grid(axis='y', alpha=0.75)

# ax = commutes.plot(grid=True,
#                    color='#607c8e' ,kind='bar', figsize=(10,4)) # bar plots are categorical
# ax.xaxis.set_visible(False) # table heading and x-labels over-printing

# plt.title('Entropy threshold of each class')
# plt.xlabel('Counts')
# plt.ylabel('Threshold')

# # add the ticks and labels to the plot
# ax.set_xticks(xticks)

# plt.grid(axis='y', alpha=0.75)
# plt.show()

In [0]:
# print(Base.entropy_th_class)

In [0]:
class Analysis():
    def __init__(self,model_loader):
        self.args =model_loader.args
        self.model_name =model_loader.model_name
        self.model = model_loader.model
        self.optimizer = model_loader.optimizer
    def Show(self, rl_img,Y_pred,X_Patch_resize,Y_Patch_resize,Y_Patch_resize_div):
        print("********Input Image*************")
        Plot_img(rl_img)

        print("********Prediction*************")
        a =np.argmax(Y_pred.T,axis=0)
        true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
        Plot_img(true_lab)
        
        print("********Patch*************")
        Plot_img(X_Patch_resize)
        
        print("********Patch Pseudo labels*************")
        a =np.argmax(Y_Patch_resize.T,axis=0)
        true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
        Plot_img(true_lab)
        
        print("********Patch Pseudo labels with division*************")
        a =np.argmax(Y_Patch_resize_div.T,axis=0)
        true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
        Plot_img(true_lab)
        
    def Final_image_results(self, num_of_samples):
        self.args.data_label_folder_target = None
        self.args.shuffel_ = False
        self.args.data_dir_target = root_base +'/dataset/cityscapes'
        self.args.data_list_target = sorted_list
        self.model.eval()
        self.model.cuda()    
        
        targetloader = CreateTrgDataLoader(self.args)
        
        print("***********Results output***********")
        
        for index, batch in enumerate(targetloader):
            print("###################################")
            if(index > num_of_samples):
                break
            
            image, _, name, rl_img = batch
            output = self.model(Variable(image).cuda())
            output = nn.functional.softmax(output, dim=1)
            rl_img = rl_img.cpu().data[0].numpy()/255.0
            output=output.cpu().data[0].numpy()
            Y_p = keras_out(output)    # later delete this line man

                    
            
            
            Y_pred = keras_out(output)
            
            ################################################
#             map_ = np.zeros((IMG_H,IMG_W))
#             SE = self_entropy(Y_pred)
            
#             map_[SE >= entropy_th] =  0
#             map_[SE < entropy_th] =  1
            
            ######################
            print(name)
            print("********Target image*************")
            Plot_img(rl_img)
            print("********whole Prediction*************")
            a =np.argmax(Y_pred.T,axis=0)
            true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
            Plot_img(true_lab)
            
            

        
    
    def Pseudo_labels_analysis(self, num_of_samples):
        self.args.data_label_folder_target = None
        self.args.shuffel_ = False
        self.args.data_dir_target = root_base +'/dataset/cityscapes'
        self.args.data_list_target = sorted_list
        self.model.eval()
        self.model.cuda()    
        
        targetloader = CreateTrgDataLoader(self.args)
        
        print("***********Testing Pseudo labels***********")
        
        for index, batch in enumerate(targetloader):
            print("###################################")
            if(index > num_of_samples):
                break
            
            image, _, name, rl_img = batch
            output = self.model(Variable(image).cuda())
            output = nn.functional.softmax(output, dim=1)
            rl_img = rl_img.cpu().data[0].numpy()/255.0
            output=output.cpu().data[0].numpy()
            Y_p = keras_out(output)    # later delete this line man
            if(balance_):
                wei = (1 - weight_mat )** d1
                output = np.multiply(output,wei)
                    
            
            
            Y_pred = keras_out(output)
            
            ################################################
            map_ = np.zeros((IMG_H,IMG_W))
            SE = self_entropy(Y_pred)
            
            map_[SE >= entropy_th] =  0
            map_[SE < entropy_th] =  1
            
            ######################
            print("********Target image*************")
            Plot_img(rl_img)
            print("********whole Prediction*************")
            a =np.argmax(Y_pred.T,axis=0)
            true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
            Plot_img(true_lab)
            print("********whole Filtering map*************")
            plt.imshow(map_.reshape(IMG_H,IMG_W),cmap='gray')
            plt.grid(False)
            plt.axis('off')
            plt.show()
            
            
            
            
            for patch in range(patches):
                

                point_X_s = 512
                point_Y_s = 256
                
                point_X = random.randint(0, IMG_W-point_X_s)
                point_Y = random.randint(0, IMG_H - point_Y_s)

            
                x_PATCH =  rl_img[ point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s,3)
                y_PATCH =  Y_pred[0, point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(1,point_Y_s,point_X_s,19)
                map_patch = map_[point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s)
   
                X_Patch_resize = resize(x_PATCH, (IMG_H, IMG_W), anti_aliasing=False)
                Y_Patch_resize = resize(y_PATCH, (1,IMG_H, IMG_W), anti_aliasing=False)
                map_patch_re = resize(map_patch, (IMG_H, IMG_W), anti_aliasing=False)
        
                

                #f.write(str(count+patch)+".png"+'\n')
                #self.save_data(X_Patch_resize, Y_Patch_resize.reshape(IMG_H,IMG_W,19), generated_data,(count+patch),map_)
                
            #count+=patches + 1
        #print("*****************END****************")
            
            
            
            
            ########################################################
                print("********patch*************")
                Plot_img(X_Patch_resize)

                print("******** patch Prediction*************")
                a =np.argmax(Y_Patch_resize.T,axis=0)
                true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
                Plot_img(true_lab)
            
#             print("********Prediction after balance*************")
#             a =np.argmax(Y_pred.T,axis=0)
#             true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
#             Plot_img(true_lab)
            
#             print("********Self entropy*************")
#             plt.imshow(SE.reshape(IMG_H,IMG_W),interpolation='nearest')
#             plt.grid(False)
#             plt.axis('off')
#             plt.show()
            
                print("******** patch Filtering map*************")
                plt.imshow(map_patch_re.reshape(IMG_H,IMG_W),cmap='gray')
                plt.grid(False)
                plt.axis('off')
                plt.show()
            
            
            
            
            
#             h = random.randint(16, 32)
#             point_X_s = int(32 * h)
#             point_Y_s = int(point_X_s/2)

#             point_X = random.randint(0, IMG_W-point_X_s)
#             point_Y = random.randint(0, IMG_H - point_Y_s)


#             x_PATCH =  rl_img[ point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s,3)
#             y_PATCH =  Y_pred[0, point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(1,point_Y_s,point_X_s,19)

#             X_Patch_resize = resize(x_PATCH, (IMG_H, IMG_W), anti_aliasing=True)
#             Y_Patch_resize = resize(y_PATCH, (1,IMG_H, IMG_W), anti_aliasing=True)
            
            
# #             map_1 = weights + epsilon   
# #             Y_pred_divide = np.divide(Y_pred,map_1)
#             output = np.divide(output,blurred)  
#             Y_pred_divide = keras_out(output)
            
#             y_PATCH_div =  Y_pred_divide[0, point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(1,point_Y_s,point_X_s,19)
#             Y_Patch_resize_div = resize(y_PATCH_div, (1,IMG_H, IMG_W), anti_aliasing=True)
            

#             self.Show(rl_img,Y_pred,X_Patch_resize,Y_Patch_resize,Y_Patch_resize_div)
        
            print("###################################")
            
    def Pseudo_labels_analysis_2(self, num_of_samples):
        self.args.data_label_folder_target = None
        self.args.shuffel_ = False
        self.args.data_dir_target = root_base +'/dataset/cityscapes'
        self.args.data_list_target = sorted_list
        self.model.eval()
        self.model.cuda()    
        
        targetloader = CreateTrgDataLoader(self.args)
        
        print("***********Testing Pseudo labels***********")
        
        for index, batch in enumerate(targetloader):
            print("###################################")
            if(index > num_of_samples):
                break
            
            image, _, name, rl_img = batch
            output = self.model(Variable(image).cuda())
            output = nn.functional.softmax(output, dim=1)
            rl_img = rl_img.cpu().data[0].numpy()/255.0
            output=output.cpu().data[0].numpy()
            Y_p = keras_out(output)    # later delete this line man
            
            image=image.cpu().data[0].numpy()
            image = np.rollaxis(image, axis = -1)
            image = np.rollaxis(image, axis = -1)
            Y_pred = keras_out(output)
            
            ################################################
            SE = self_entropy(Y_pred)
            ########################################################
            Plot_img(rl_img)
          
            print("******** full image Prediction*************")
            a =np.argmax(Y_pred.T,axis=0)
            true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
            Plot_img(true_lab)
            
            print("********Self entropy*************")
            plt.imshow(SE.reshape(IMG_H,IMG_W),interpolation='nearest')
            plt.grid(False)
            plt.axis('off')
            plt.show()
            #################################################
            rl_patch = []
            se_Main = []
            se_Patch = []
            se_diff=[]
            for patch in range(patches):
                

                point_X_s = 512
                point_Y_s = 256
                
                point_X = random.randint(0, IMG_W-point_X_s)
                point_Y = random.randint(0, IMG_H - point_Y_s)

                real_patch =  rl_img[ point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s,3)
               
                x_PATCH =  image[ point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(point_Y_s,point_X_s,3)
                y_PATCH =  Y_pred[0, point_Y:point_Y+point_Y_s, point_X:point_X+point_X_s].reshape(1,point_Y_s,point_X_s,19)
               
                X_Patch_resize = resize(x_PATCH, (IMG_H, IMG_W), anti_aliasing=False)
                Y_Patch_resize = resize(y_PATCH, (1,IMG_H, IMG_W), anti_aliasing=False)
                
                real_patch_resize = resize(real_patch, (IMG_H, IMG_W), anti_aliasing=False)
                
                X_Patch_resize = np.rollaxis(X_Patch_resize, axis = -1).reshape(1,3,IMG_H,IMG_W)
                X_Patch_resize = torch.from_numpy(X_Patch_resize).float().to(device)
              
                
                output = self.model(Variable(X_Patch_resize).cuda())
                output = nn.functional.softmax(output, dim=1)
                output=output.cpu().data[0].numpy()
                y_patch_out = keras_out(output)
                
                SE_main = self_entropy(Y_Patch_resize)
                SE_patch = self_entropy(y_patch_out)
                
                diff = (np.mean(SE_patch) - np.mean(SE_main))
                
                
                
                rl_patch.append(real_patch_resize)
                se_Main.append(SE_main)
                se_Patch.append(SE_patch)
                se_diff.append(diff)
                
            se_diff = np.array(se_diff)
            arg_max =np.argmax(se_diff)
            maxi = np.max(se_diff)
            print(maxi)
            
            Plot_img(rl_patch[arg_max])
#             print("********Self entropy real*************")
#             plt.imshow(se_Main[arg_max].reshape(IMG_H,IMG_W),interpolation='nearest')
#             plt.grid(False)
#             plt.axis('off')
#             plt.show()

            print("******** patch Prediction*************")
            a =np.argmax(y_patch_out.T,axis=0)
            true_lab = visualize(a.T.reshape((IMG_H,IMG_W)), False)
            Plot_img(true_lab)
            
            print("********Self entropy patch*************")
            plt.imshow(se_Patch[arg_max].reshape(IMG_H,IMG_W),interpolation='nearest')
            plt.grid(False)
            plt.axis('off')
            plt.show()
            
            


            print("######################################################")
        
           
                

            
    def Analyze_class_frequency(self,num_of_samples ):
        self.args.data_label_folder_target = None
        self.args.shuffel_ = False
        self.args.data_dir_target = root_base +'/dataset/cityscapes'
        self.args.data_list_target = sorted_list
        self.model.eval()
        self.model.cuda()    
        
        targetloader = CreateTrgDataLoader(self.args)
        
        print("***********Testing Pseudo labels***********")
        
        class_freq = 0
        
        for index, batch in enumerate(targetloader):
            
            if(index > num_of_samples):
                break
            
            image, _, name, rl_img = batch
            output = self.model(Variable(image).cuda())
            output = nn.functional.softmax(output, dim=1)
            rl_img = rl_img.cpu().data[0].numpy()/255.0
            output=output.cpu().data[0].numpy()
            Y_p = keras_out(output)    # later delete this line man
            
            image=image.cpu().data[0].numpy()
            image = np.rollaxis(image, axis = -1)
            image = np.rollaxis(image, axis = -1)
            Y_pred = keras_out(output)
            
            class_are =   np.mean(Y_pred,axis = (0,1,2))
            
            class_freq += class_are
        
        return class_freq/num_of_samples
            
            
        
                

            
            
            
        

In [0]:
# Base.sorting()

In [0]:
entropy_th = 0.4
patches = 5
balance_  = False

In [0]:
ana =Analysis(model_initl)

In [0]:
ana.Final_image_results( 82)

block diagram image select

In [0]:
ana.Pseudo_labels_analysis(5)

In [0]:
ana.Pseudo_labels_analysis_2(7)

In [0]:
ana.Pseudo_labels_analysis_2(7)

In [0]:


class_=ana.Analyze_class_frequency(100
                                  )


In [0]:
class_